In [1]:
#Analysis of BlackFriday Data

import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn import linear_model
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error
import xgboost as xgb

import matplotlib as plt
import seaborn as sns

C:\Users\Muhammad Bilawal\Anaconda2\envs\pdap2018\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data_train       = pd.read_csv(r'E:\Data_science\project_BlackFriday\train.csv')
data_test        = pd.read_csv(r'E:\Data_science\project_BlackFriday\test.csv')
data_submission  = pd.read_csv(r'E:\Data_science\project_BlackFriday\Sample_Submission_Tm9Lura.csv')

In [3]:
data_train.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [4]:
data_test.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,NaN
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,NaN
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,NaN
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,NaN
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0


In [5]:
data_submission.head() 

,User_ID,Product_ID,Purchase
0,1000004,P00128942,9000
1,1000009,P00113442,9000
2,1000010,P00288442,9000
3,1000010,P00145342,9000
4,1000011,P00053842,9000


In [6]:
concat = [data_train,data_test]

val = pd.concat(concat)

val.shape

(783667, 12)

In [7]:
val.head()

,Age,City_Category,Gender,Marital_Status,Occupation,Product_Category_1,Product_Category_2,Product_Category_3,Product_ID,Purchase,Stay_In_Current_City_Years,User_ID
0,0-17,A,F,0,10,3,NaN,NaN,P00069042,8370.0,2,1000001
1,0-17,A,F,0,10,1,6.0,14.0,P00248942,15200.0,2,1000001
2,0-17,A,F,0,10,12,NaN,NaN,P00087842,1422.0,2,1000001
3,0-17,A,F,0,10,12,14.0,NaN,P00085442,1057.0,2,1000001
4,55+,C,M,0,16,8,NaN,NaN,P00285442,7969.0,4+,1000002


In [8]:
val.dtypes

Age                            object
City_Category                  object
Gender                         object
Marital_Status                  int64
Occupation                      int64
Product_Category_1              int64
Product_Category_2            float64
Product_Category_3            float64
Product_ID                     object
Purchase                      float64
Stay_In_Current_City_Years     object
User_ID                         int64
dtype: object

In [9]:
val.isnull().any()

Age                           False
City_Category                 False
Gender                        False
Marital_Status                False
Occupation                    False
Product_Category_1            False
Product_Category_2             True
Product_Category_3             True
Product_ID                    False
Purchase                       True
Stay_In_Current_City_Years    False
User_ID                       False
dtype: bool

In [10]:
val.fillna(900, inplace=True)

In [11]:
val.isnull().any()

Age                           False
City_Category                 False
Gender                        False
Marital_Status                False
Occupation                    False
Product_Category_1            False
Product_Category_2            False
Product_Category_3            False
Product_ID                    False
Purchase                      False
Stay_In_Current_City_Years    False
User_ID                       False
dtype: bool

In [12]:
purchase = val.Purchase

In [13]:
purchase

0          8370.0
1         15200.0
2          1422.0
3          1057.0
4          7969.0
5         15227.0
6         19215.0
7         15854.0
8         15686.0
9          7871.0
10         5254.0
11         3957.0
12         6073.0
13        15665.0
14         5378.0
15         2079.0
16        13055.0
17         8851.0
18        11788.0
19        19614.0
20         8584.0
21         9872.0
22         9743.0
23         5982.0
24        11927.0
25        16662.0
26         5887.0
27         6973.0
28         5391.0
29        16352.0
           ...   
233569      900.0
233570      900.0
233571      900.0
233572      900.0
233573      900.0
233574      900.0
233575      900.0
233576      900.0
233577      900.0
233578      900.0
233579      900.0
233580      900.0
233581      900.0
233582      900.0
233583      900.0
233584      900.0
233585      900.0
233586      900.0
233587      900.0
233588      900.0
233589      900.0
233590      900.0
233591      900.0
233592      900.0
233593    

In [14]:
purchase = np.array(purchase)
purchase

array([  8370.,  15200.,   1422., ...,    900.,    900.,    900.])

In [15]:
val.drop(["Purchase"], inplace=True)
val

,Age,City_Category,Gender,Marital_Status,Occupation,Product_Category_1,Product_Category_2,Product_Category_3,Product_ID,Purchase,Stay_In_Current_City_Years,User_ID
0,0-17,A,F,0,10,3,900.0,900.0,P00069042,8370.0,2,1000001
1,0-17,A,F,0,10,1,6.0,14.0,P00248942,15200.0,2,1000001
2,0-17,A,F,0,10,12,900.0,900.0,P00087842,1422.0,2,1000001
3,0-17,A,F,0,10,12,14.0,900.0,P00085442,1057.0,2,1000001
4,55+,C,M,0,16,8,900.0,900.0,P00285442,7969.0,4+,1000002
5,26-35,A,M,0,15,1,2.0,900.0,P00193542,15227.0,3,1000003
6,46-50,B,M,1,7,1,8.0,17.0,P00184942,19215.0,2,1000004
7,46-50,B,M,1,7,1,15.0,900.0,P00346142,15854.0,2,1000004
8,46-50,B,M,1,7,1,16.0,900.0,P0097242,15686.0,2,1000004
9,26-35,A,M,1,20,8,900.0,900.0,P00274942,7871.0,1,1000005


In [16]:
val.dtypes

Age                            object
City_Category                  object
Gender                         object
Marital_Status                  int64
Occupation                      int64
Product_Category_1              int64
Product_Category_2            float64
Product_Category_3            float64
Product_ID                     object
Purchase                      float64
Stay_In_Current_City_Years     object
User_ID                         int64
dtype: object

In [17]:
val = val.applymap(str)
val.dtypes

Age                           object
City_Category                 object
Gender                        object
Marital_Status                object
Occupation                    object
Product_Category_1            object
Product_Category_2            object
Product_Category_3            object
Product_ID                    object
Purchase                      object
Stay_In_Current_City_Years    object
User_ID                       object
dtype: object

In [18]:
val = np.array(val)

for i in range(val.shape[1]):
    x = preprocessing.LabelEncoder()
    x.fit(list(val[:,i]))
    val[:, i] = x.transform(val[:, i])

In [19]:
val = val.astype(int)

In [20]:
#splitting the data from Purchase column and rest of the data stored in val
val_np = val[:data_train.shape[0],:]
purchase_np= purchase[:data_train.shape[0]]

first_level_rows = np.random.randint(data_train.shape[0], size = np.int(data_train.shape[0]/2))

val_fl = val_np[first_level_rows,:]
purchase_fl = purchase[first_level_rows]

val_sl = val_np[-first_level_rows,:]
purchase_sl = purchase_np[-first_level_rows]

In [21]:
val_fl.shape, purchase_fl.shape, val_sl.shape, purchase_sl.shape

((275034, 12), (275034,), (275034, 12), (275034,))

In [22]:
val_fl

array([[    3,     1,     0, ..., 15505,     1,  5043],
       [    2,     1,     1, ...,  8342,     3,  1782],
       [    2,     1,     1, ..., 13983,     1,   783],
       ..., 
       [    2,     0,     1, ..., 12462,     3,  2006],
       [    3,     1,     1, ..., 13660,     1,   259],
       [    3,     2,     1, ..., 16170,     3,  2076]])

In [23]:
purchase_sl

array([  3117.,   5453.,  13553., ...,  15393.,  13135.,  11896.])

In [24]:
xgModel = xgb.DMatrix(val_fl,label=purchase_fl)
viewlist=[(xgModel, 'train')]

In [25]:
#model1
params = {}
params["min_child_weight"] = 10
params["subsample"] = 0.7
params["colsample_bytree"] = 0.7
params["scale_pos_weight"] = 0.8
params["silent"] = 1
params["max_depth"] = 6
params["nthread"] = 6
#params["gamma"] = 1
params["objective"] = "reg:linear"
params["eta"] = 0.1
params["base_score"] = 1800
params["eval_metric"] = "rmse"
params["seed"] = 0

plst = list(params.items())
num_rounds = 3000

model_1 = xgb.train(plst,xgModel,num_rounds)

In [26]:
#model2
params = {}
params["min_child_weight"] = 10
params["subsample"] = 0.7
params["colsample_bytree"] = 0.7
params["scale_pos_weight"] = 0.8
params["silent"] = 1
params["max_depth"] = 8
params["nthread"] = 6
#params["gamma"] = 1
params["objective"] = "reg:linear"
params["eta"] = 0.1
params["base_score"] = 1800
params["eval_metric"] = "rmse"
params["seed"] = 0

plst = list(params.items())
num_rounds = 1420

model_2 = xgb.train(plst, xgModel, num_rounds)

In [27]:
#model 3 

params = {}
params["min_child_weight"] = 10
params["subsample"] = 0.7
params["colsample_bytree"] = 0.7
params["scale_pos_weight"] = 0.8
params["silent"] = 1
params["max_depth"] = 10
params["nthread"] = 6
#params["gamma"] = 1
params["objective"] = "reg:linear"
params["eta"] = 0.1
params["base_score"] = 1800
params["eval_metric"] = "rmse"
params["seed"] = 0

plst = list(params.items())
num_rounds = 1200

model_3 = xgb.train(plst, xgModel, num_rounds)

In [28]:
#model 4 

params = {}
params["min_child_weight"] = 10
params["subsample"] = 0.7
params["colsample_bytree"] = 0.7
params["scale_pos_weight"] = 0.8
params["silent"] = 1
params["max_depth"] = 12
params["nthread"] = 6
#params["gamma"] = 1
params["objective"] = "reg:linear"
params["eta"] = 0.1
params["base_score"] = 1800
params["eval_metric"] = "rmse"
params["seed"] = 0

plst = list(params.items())
num_rounds = 800
model_4 = xgb.train(plst, xgModel, num_rounds)

In [29]:
# This set of models will be ExtraTrees

In [30]:
# model 5
model_5 = ExtraTreesRegressor(n_estimators=1450, max_depth=8,min_samples_split=10, min_samples_leaf=10, oob_score=True, n_jobs=6, random_state=123, verbose=1, bootstrap=True)
model_5.fit(val_fl, purchase_fl)

[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:   13.6s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:  1.1min
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:  2.6min
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:  4.7min
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:  7.1min
[Parallel(n_jobs=6)]: Done 1450 out of 1450 | elapsed:  8.1min finished


ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=8,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=10, min_samples_split=10,
          min_weight_fraction_leaf=0.0, n_estimators=1450, n_jobs=6,
          oob_score=True, random_state=123, verbose=1, warm_start=False)

In [31]:
#model 6
model_6 = ExtraTreesRegressor(n_estimators=3000, max_depth=6,min_samples_split=10, min_samples_leaf=10, oob_score=True, n_jobs=6, random_state=123, verbose=1, 
bootstrap=True)
model_6.fit(val_fl, purchase_fl)

[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    8.7s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:   43.7s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:  1.7min
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:  3.2min
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:  5.0min
[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed:  7.3min
[Parallel(n_jobs=6)]: Done 2438 tasks      | elapsed: 10.1min
[Parallel(n_jobs=6)]: Done 3000 out of 3000 | elapsed: 12.2min finished


ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=6,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=10, min_samples_split=10,
          min_weight_fraction_leaf=0.0, n_estimators=3000, n_jobs=6,
          oob_score=True, random_state=123, verbose=1, warm_start=False)

In [32]:
#model 7
model_7 = ExtraTreesRegressor(n_estimators=800, max_depth=12, min_samples_split=10, min_samples_leaf=10, oob_score=True, n_jobs=6, random_state=123, verbose=1, 
bootstrap=True)
model_7.fit(val_fl, purchase_fl)

[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:   14.5s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:  1.1min
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:  2.6min
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:  4.8min
[Parallel(n_jobs=6)]: Done 800 out of 800 | elapsed:  4.8min finished


ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=12,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=10, min_samples_split=10,
          min_weight_fraction_leaf=0.0, n_estimators=800, n_jobs=6,
          oob_score=True, random_state=123, verbose=1, warm_start=False)

In [33]:
#This set of models will be RandomForest 

In [34]:
#model 8:
model_8 = RandomForestRegressor(n_estimators=3000, max_depth=6, oob_score=True, n_jobs=6, random_state=123, min_samples_split=10, min_samples_leaf=10)
model_8.fit(val_fl, purchase_fl)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=6,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=10, min_samples_split=10,
           min_weight_fraction_leaf=0.0, n_estimators=3000, n_jobs=6,
           oob_score=True, random_state=123, verbose=0, warm_start=False)

In [35]:
#model 9:
model_9 = RandomForestRegressor(n_estimators=1500, max_depth=8, oob_score=True, n_jobs=6, random_state=123, min_samples_split=10, min_samples_leaf=10)
model_9.fit(val_fl, purchase_fl)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=8,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=10, min_samples_split=10,
           min_weight_fraction_leaf=0.0, n_estimators=1500, n_jobs=6,
           oob_score=True, random_state=123, verbose=0, warm_start=False)

In [36]:
#model10;
model_10 = RandomForestRegressor(n_estimators=800, max_depth=12, oob_score=True, n_jobs=6, random_state=123, min_samples_split=10, min_samples_leaf=10)
model_10.fit(val_fl, purchase_fl)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=12,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=10, min_samples_split=10,
           min_weight_fraction_leaf=0.0, n_estimators=800, n_jobs=6,
           oob_score=True, random_state=123, verbose=0, warm_start=False)

In [37]:
model_1_predict = model_1.predict(xgb.DMatrix(val_sl))
model_2_predict = model_2.predict(xgb.DMatrix(val_sl))
model_3_predict = model_3.predict(xgb.DMatrix(val_sl))
model_4_predict = model_4.predict(xgb.DMatrix(val_sl))
model_5_predict = model_5.predict(val_sl)
model_6_predict = model_6.predict(val_sl)
model_7_predict = model_7.predict(val_sl)
model_8_predict = model_8.predict(val_sl)
model_9_predict = model_9.predict(val_sl)
model_10_predict = model_10.predict(val_sl)

[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.4s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    1.8s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:    4.2s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:    7.3s
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:   11.3s
[Parallel(n_jobs=6)]: Done 1450 out of 1450 | elapsed:   13.1s finished
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.2s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    1.0s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:    2.5s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:    4.9s
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:    7.9s
[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed:   11.7s
[Parallel(n_jobs=6)]: Done 2438 tasks      | elapsed:   15.8s
[Parallel(n_jobs=6)]: Done 3000 out of 3000 | elapsed:   19.5s finished
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.5s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    2.9

In [38]:
val_sl_w_meta = np.concatenate((val_sl, np.vstack((model_1_predict, model_2_predict, model_3_predict, model_4_predict, model_5_predict,model_6_predict, model_7_predict, model_8_predict, model_9_predict, model_10_predict)).T), axis=1)

In [39]:
val_sl_w_meta

array([[  2.00000000e+00,   0.00000000e+00,   1.00000000e+00, ...,
          3.15222531e+03,   3.14120386e+03,   3.11643058e+03],
       [  4.00000000e+00,   2.00000000e+00,   1.00000000e+00, ...,
          5.22233932e+03,   5.37623956e+03,   5.45304496e+03],
       [  4.00000000e+00,   0.00000000e+00,   1.00000000e+00, ...,
          1.21395297e+04,   1.36549549e+04,   1.35242438e+04],
       ..., 
       [  2.00000000e+00,   0.00000000e+00,   1.00000000e+00, ...,
          1.53690258e+04,   1.53740229e+04,   1.53927985e+04],
       [  0.00000000e+00,   2.00000000e+00,   0.00000000e+00, ...,
          1.23413136e+04,   1.36845336e+04,   1.31062036e+04],
       [  5.00000000e+00,   1.00000000e+00,   1.00000000e+00, ...,
          1.18095969e+04,   1.18939088e+04,   1.18956559e+04]])

In [40]:
#second level model with meta features

kfolds = KFold(val_sl_w_meta.shape[0], n_folds=5)

In [41]:
params = {}
params["min_child_weight"] = 10
params["subsample"] = 0.7
params["colsample_bytree"] = 0.7
params["scale_pos_weight"] = 0.8
params["silent"] = 1
params["max_depth"] = 8
params["nthread"] = 6
#params["gamma"] = 1
params["objective"] = "reg:linear"
params["eta"] = 0.1
params["base_score"] = 1800
params["eval_metric"] = "rmse"
params["seed"] = 0

plst = list(params.items())
num_rounds = 1400

In [59]:
for val_index, validation_index in kfolds:
    
    val_X, validation_X = val_sl_w_meta[val_index, :], val_sl_w_meta[validation_index, :]
    val_y, validation_y = val_sl[val_index], purchase_sl[validation_index]
    
    xgtrain = xgb.DMatrix(val_X, label=val_y)
    watchlist = [(xgtrain, 'val')]
    model_cv_xgboost = xgb.train(plst, xgtrain, num_rounds)
    model_cv_predict = model_cv_xgboost.predict(xgb.DMatrix(validation_X))
    np.sqrt(mean_squared_error(validation_y, model_cv_predict))

In [61]:
print(np.sqrt(mean_squared_error(validation_y, model_cv_predict)))

10539.1471442


In [62]:
xgtrain = xgb.DMatrix(val_sl_w_meta, label=purchase_sl)
watchlist = [(xgtrain, 'train')]
model_ss_xgboost = xgb.train(plst, xgtrain, num_rounds)

In [63]:
#Final prediction on dataset

In [68]:
model_1_predict = model_1.predict(xgb.DMatrix(val[data_train.shape[0]:, :]))
model_2_predict = model_2.predict(xgb.DMatrix(val[data_train.shape[0]:, :]))
model_3_predict = model_3.predict(xgb.DMatrix(val[data_train.shape[0]:, :]))
model_4_predict = model_4.predict(xgb.DMatrix(val[data_train.shape[0]:, :]))
model_5_predict = model_5.predict(val[data_train.shape[0]:, :])
model_6_predict = model_6.predict(val[data_train.shape[0]:, :])
model_7_predict = model_7.predict(val[data_train.shape[0]:, :])
model_8_predict = model_8.predict(val[data_train.shape[0]:, :])
model_9_predict = model_9.predict(val[data_train.shape[0]:, :])
model_10_predict = model_10.predict(val[data_train.shape[0]:, :])

[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    1.0s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:    2.6s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:    4.7s
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:    7.4s
[Parallel(n_jobs=6)]: Done 1450 out of 1450 | elapsed:    8.6s finished
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.8s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:    2.1s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:    3.9s
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:    5.8s
[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed:    8.2s
[Parallel(n_jobs=6)]: Done 2438 tasks      | elapsed:   11.3s
[Parallel(n_jobs=6)]: Done 3000 out of 3000 | elapsed:   14.0s finished
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.2s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    1.4

In [70]:
test_sl_w_meta = np.concatenate((val[data_train.shape[0]:, :], np.vstack((model_1_predict, model_2_predict, model_3_predict, model_4_predict, model_5_predict,model_6_predict, model_7_predict, model_8_predict, model_9_predict, model_10_predict)).T), axis=1)

In [71]:
model_sl_predict = model_ss_xgboost.predict(xgb.DMatrix(test_sl_w_meta))

In [72]:
np.max(model_sl_predict), np.min(model_sl_predict)

(1800.0, 1800.0)